In [ ]:
# Dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc
from flask import Flask, jsonify, render_template, request
import csv

In [ ]:
#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///DataSets/belly_button_biodiversity.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
Otu = Base.classes.otu
Samples = Base.classes.samples
Samples_metadata = Base.classes.samples_metadata

# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Create an empty list for sample ids
sample_ids = []

# Query results from matadata table
results = session.query(Samples_metadata.SAMPLEID)

# Loop through the query results and append the list with sample ids
for result in results:
    sample_ids.append("BB_" + str(result[0]))
    
sample_ids

In [ ]:

# Create an empty list for otu description
otu_desc = []

# Query results from otu table
results = session.query(Otu.lowest_taxonomic_unit_found)

# Loop through the query results and append the list with otu description
for result in results:
    otu_desc.append(result[0])

otu_desc

In [ ]:
def metadata(sample):
    sample_id = sample.replace("BB_","")

    metadata = session.query(Samples_Metadata.AGE, Samples_Metadata.BBTYPE, Samples_Metadata.ETHNICITY, Samples_Metadata.GENDER, Samples_Metadata.LOCATION, Samples_Metadata.SAMPLEID).filter_by(SAMPLEID=sample).first()
    metadict = {"AGE":metadata[0],"BBTYPE":metadata[1],"ETHNICITY":metadata[2], "GENDER":metadata[3],"LOCATION":metadata[4],"SAMPLEID":metadata[5]}
    return jsonify(metadict)

In [ ]:

def wfreq(sample):
    sample = sample.replace("BB_","")

    # Query from metadata table
    wfreq = session.query(Samples_Metadata.WFREQ).filter_by(SAMPLEID = sample).scalar()
    
    return str(wfreq)

In [ ]:

def samples(sample):

    # Create a sample query
    sample_query = "Samples." + sample

    # Create an empty dictionary and list
    sample_info = {}
    otu_ids = []
    sample_values = []

    # Create a query 
    results = session.query(Samples.otu_id, sample_query).order_by(desc(sample_query))

    # Loop through the results and append otu_ids and sample_values lists
    for result in results:
        otu_ids.append(result[0])
        sample_values.append(result[1])

    # Add these values to the dictionary
    sample_info = [{
        "otu_ids": otu_ids,
        "sample_values": sample_values
    }]

    return jsonify(sample_info)